Segundo aproach:
- Mejoramos la funcion para clasificar en los buckets.
- Entrenamos con 100000 iteraciones y corrimos 10000 veces, los valores fluctuaron entre 10 y 37.

In [1]:
#pip install gym pygame
import gym
import time
from random import randint
# imports personales
import numpy as np

env = gym.make('CartPole-v1')

def epsilon_greedy_policy(state, Q, epsilon):
    if np.random.random() < epsilon:
        action = randint(0,1)
    else:
        action = np.argmax(Q[state])
    return action

def optimal_policy(state, Q):
    action = np.argmax(Q[state])
    return action

BINS_SIZE = 20

bins = np.linspace(-0.2095, 0.2095, BINS_SIZE) #armamos los bins en los q clasificar observaciones

def get_state(obs_radang):
    return np.digitize(obs_radang, bins)#nos dice en que bin cae cada observacion

# Tenemos que crear Q con un lugar mas ya que tendremos rango desde el 0 al 20.
Q = np.zeros((BINS_SIZE + 1,2))

#defino params de entrenamiento
epsilon = 0.9
discount_factor = 0.9
learning_rate = 0.9

def update_q(state, action, reward, new_state):
        """
        Updates Q-table using the rule as described by Sutton and Barto in
        Reinforcement Learning.
        """
        Q[state][action] += (learning_rate * (reward + discount_factor * np.max(Q[new_state]) - Q[state][action]))

for episode in range(100000):
    obs = env.reset()
    current_state = get_state(obs[2])
    done = False
    while not done:
        #elijo acccion
        action = epsilon_greedy_policy(current_state, Q, epsilon)
        #actualizo accion
        obs, reward, done, info = env.step(action)
        new_state = get_state(obs[2])
        #actualizo q
        update_q(current_state, action, reward, new_state)
        current_state = new_state
print("Me entrene")
print(Q)

def simulate_random_agent(number_of_samples):
    rewards = [0] * number_of_samples #inicializo lista para almacenar recompensas
    for index in range(number_of_samples):
        obs = env.reset()
        done = False
        while not done:
            policy = optimal_policy(get_state(obs[2]), Q)
            obs, reward, done, info = env.step(policy)
            rewards[index] += reward
            if done:
                env.reset()
    env.close()
    average = np.average(rewards)

    return average, rewards

print("Simulacion aleatoria 10000 muestreos")
start = time.time()
print("Valor esperado :")
print(simulate_random_agent(10000)[0])
end = time.time()
print("------- Timetaken(seconds) ----------------")
print(end-start)


Me entrene
[[0.         0.        ]
 [3.0786019  2.46616535]
 [1.97501708 1.90023242]
 [2.73142959 2.73844591]
 [3.49641095 3.76768719]
 [5.85799048 5.9029559 ]
 [4.57868082 8.56812956]
 [8.58716126 8.82993095]
 [9.98764289 9.92948683]
 [9.98960901 9.99077745]
 [9.96455203 9.97226995]
 [9.96687146 8.70012727]
 [8.42888468 8.36748461]
 [8.37417973 8.2558867 ]
 [6.29100257 5.59267858]
 [5.33345762 4.5544819 ]
 [3.59181301 3.70367154]
 [3.50644211 2.00379661]
 [1.91333233 1.91640434]
 [1.00818994 1.00001071]
 [0.         0.        ]]
Simulacion aleatoria 10000 muestreos
Valor esperado :
10.9401
------- Timetaken(seconds) ----------------
4.959854602813721
